In [1]:
import os
import pickle
import gc

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2

from tensorflow_probability import distributions as tfd

from sklearn.model_selection import train_test_split

In [2]:
MODEL = "8-lstm-future-covariates-mdn"
# COVARIATE_COLUMNS = ['covs_week_sin','covs_week_cos','covs_day_sin','covs_day_cos','covs_hour_sin','covs_hour_cos','covs_rbf_month_1','covs_rbf_month_2','covs_rbf_month_3','covs_rbf_month_4','covs_rbf_month_5','covs_rbf_month_6','covs_rbf_month_7','covs_rbf_month_8','covs_rbf_month_9','covs_rbf_month_10','covs_rbf_month_11','covs_rbf_month_12']
COVARIATE_COLUMNS = ['covs_week_sin','covs_week_cos','covs_day_sin','covs_day_cos','covs_hour_sin','covs_hour_cos']
config = {
    "N_PREV": 120,
    "N_FORWARD": 48,
    "STEP_SIZE": 1,
    "N_PROB": 1,
    
    "LATENT_SIZE": 16,
    "DENSE_SIZE": 16,
    
    "LR": 0.001,
    "BATCH_SIZE": 512,
    "EPOCHS": 50,
    
    "L1": 0.0000001, 
    "L2": 0.0000001,
    
    "TARGET_COL": 'normalized_level',
}

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)

wandb: Currently logged in as: kmcguigan. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/kiernan/.netrc


True

## **Load Data Function**

In [4]:
def get_samples(df, set_number):
    subset = df.loc[df.set==set_number, :]
    shape = subset.shape
    if(shape[0] > 100_000):
        subset = subset.iloc[-100_000:, :]
    data = subset[config['TARGET_COL']].values
    covariates = subset[COVARIATE_COLUMNS].values
    print(f'Data Shape: {shape}, Reduced Shape {data.shape}')
    
    X, y, future_covs, dates = [], [], [], []
    for start_idx in range(0, data.shape[0]-config["N_PREV"]-config["N_FORWARD"], config["STEP_SIZE"]):
        mid_idx = start_idx + config["N_PREV"]
        end_idx = mid_idx + config["N_FORWARD"]
        
        mus = np.zeros((config["N_PREV"],config["N_PROB"]))
        stds = np.zeros((config["N_PREV"],config["N_PROB"]))
        weights = np.zeros((config["N_PREV"],config["N_PROB"]))
        mus[:, 0] = data[start_idx:mid_idx]
        weights[:, 0] = 1.0
        
        X_data = np.stack([mus, stds, weights], axis=1)
        X.append(X_data)
        y.append(data[mid_idx:end_idx])
        future_covs.append(covariates[mid_idx:end_idx])
        dates.append(subset.iloc[mid_idx:end_idx].index)
        
    y = np.array(y)
    y = np.expand_dims(y, axis=-1)
    return np.array(X), y, np.array(future_covs), np.array(dates)

def to_dataset(X1, X2, y):
    dataset = tf.data.Dataset.from_tensor_slices(({'series':X1, 'future_dates':X2, 'targets':y}, y))
    dataset = dataset.cache().shuffle(X1.shape[0]+1).batch(config["BATCH_SIZE"]).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def get_train_and_val_datasets(X1, X2, y):
    X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(X1, X2, y, test_size=0.15, shuffle=True)
    print(f"Train shape: {X1_train.shape}, {X2_train.shape}, {y_train.shape}")
    print(f"Val shape: {X1_val.shape}, {X2_val.shape}, {y_val.shape}")
    train_ds = to_dataset(X1_train, X2_train, y_train)
    val_ds = to_dataset(X1_val, X2_val, y_val)
    return train_ds, val_ds

def df_to_samples(df):
    X, y, future, _ = get_samples(df, 0)
    train_ds, val_ds = get_train_and_val_datasets(X, future, y)
    X_test, y_test, future_test, dates = get_samples(df, 1)
    print(f"Test shape: {X_test.shape}, {future_test.shape}, {y_test.shape}, {dates.shape}")
    return train_ds, val_ds, X_test, y_test, future_test, dates
    
def get_data(fold):
    df = pd.read_csv(f'data/fold{fold}_data_v3.csv')
    with open(f'./data/fold{fold}_normalizer_v3.pkl', mode='rb') as f:
        normalizer = pickle.load(f)
    return df_to_samples(df), normalizer

args, _ = get_data(1)
del args
gc.collect()

Data Shape: (159096, 23), Reduced Shape (100000,)
Train shape: (84857, 120, 3, 1), (84857, 48, 6), (84857, 48, 1)
Val shape: (14975, 120, 3, 1), (14975, 48, 6), (14975, 48, 1)
Metal device set to: Apple M1 Pro
Data Shape: (43390, 23), Reduced Shape (43390,)


2022-07-04 11:33:04.730935: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-04 11:33:04.731088: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Test shape: (43222, 120, 3, 1), (43222, 48, 6), (43222, 48, 1), (43222, 48)


0

In [5]:
# gm = tfd.MixtureSameFamily(
#     mixture_distribution=tfd.Categorical(probs=[[0.4,0.6],[0.4,0.6],[0.4,0.6]]),
#     components_distribution=tfd.Normal(
#         loc=[[2, 3],
#              [4, 5],
#              [2, 3]],       
#         scale=[[1, 1],
#                [2, 2],
#                [1, 1]]
#     )
# )
    
# print(gm.batch_shape, gm.event_shape)
# print(gm.mean())
# print(gm.variance())
# y = np.array([[5], [5], [5]])
# log_likelihood = gm.log_prob(y.T)   
# print(log_likelihood)
# print(-np.mean(log_likelihood, axis=-1))

## **Model Functions**

In [6]:
def nnelu(input):
    """ Computes the Non-Negative Exponential Linear Unit
    """
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

tf.keras.utils.get_custom_objects().update({'nnelu': tf.keras.layers.Activation(nnelu)})

def identity(y_true, y_pred):
    return K.mean(y_pred)

def variational_loss(y_true, mu, sigma):
    inv_sigma_2 = 1 / K.square(sigma + K.epsilon())
    phi = inv_sigma_2 * K.exp(-inv_sigma_2 * K.square(y_true - mu))
    return -K.log(phi + K.epsilon())

def mdn_loss(y_true, mu, sigma, alpha):
    print("mu: ", mu.shape)
    print("sigma: ", sigma.shape)
    print("alpha: ", alpha.shape)
    inv_sigma_2 = 1 / K.square(sigma + K.epsilon())
    print("inv_sigma_2: ", inv_sigma_2.shape)
    phi = inv_sigma_2 * K.exp(-inv_sigma_2 * K.square(y_true - mu))
    print("phi: ", phi.shape)
    loss = -K.log(phi + K.epsilon())
    loss *= alpha
    print("loss: ", loss.shape)
    return loss

def gnll_loss(y, mu, sigma, alpha):
    """ Computes the mean negative log-likelihood loss of y given the mixture parameters.
    """    
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Normal(
            loc=mu,       
            scale=sigma
        )
    )
    
#     print('mu shape: ', mu)
    log_likelihood = gm.log_prob(tf.transpose(y))   
#     print('log likelihood: ', log_likelihood) # Evaluate log-probability of y
    return -tf.reduce_mean(log_likelihood, axis=-1)

def gnll_metric(y, mu, sigma, alpha):
    """ Computes the mean negative log-likelihood loss of y given the mixture parameters.
    """    
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Normal(
            loc=mu,       
            scale=sigma
        )
    )
    
    probability = gm.prob(tf.transpose(y))                 # Evaluate log-probability of y
    
    return tf.reduce_mean(probability, axis=-1)

# Test where sigma depends on the predicted mu concat with the input data

def get_model():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 3, config['N_PROB']), name='series')
    future_covs = tf.keras.layers.Input(shape=(config["N_FORWARD"], len(COVARIATE_COLUMNS)), name='future_dates')
    targets = tf.keras.layers.Input(shape=(config["N_FORWARD"], 1), name='targets')
    
    reshaper = tf.keras.layers.Reshape((1, 3*config['N_PROB']))
    lstm1 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm2 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm3 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=False, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    dense1 = tf.keras.layers.Dense(config["DENSE_SIZE"])
    dense2 = tf.keras.layers.Dense(config["DENSE_SIZE"]//2)
    
    # create the mdn layers
    mu_layer = tf.keras.layers.Dense(config['N_PROB'], name="mu")
    sigma_layer = tf.keras.layers.Dense(config['N_PROB'], name="sigma", activation="softplus")
    alpha_layer = tf.keras.layers.Dense(config['N_PROB'], name="alpha", activation="softmax")
    loss_layer = tf.keras.layers.Lambda(lambda x: gnll_loss(x[0], x[1], x[2], x[3]), name="loss")
    
    forecasts = []
    losses = []
    log_probabilities = []
    
    x = tf.keras.layers.Reshape((config["N_PREV"], 3*config['N_PROB']))(inputs)
    x, hidden_state1, cell_state1 = lstm1(x)
    x, hidden_state2, cell_state2 = lstm2(x)
    x, hidden_state3, cell_state3 = lstm3(x)
    x = K.concatenate([x, future_covs[:, 0, :]])
    x = dense1(x)
    x = dense2(x)
    
    mu = mu_layer(x)
    sigma = sigma_layer(x)
    alpha = alpha_layer(x)
    
    stacked_prediction = tf.stack([mu, sigma, alpha], axis=1)
#     print("stacked_predictions: ", stacked_prediction.type_spec.shape)
    
    forecasts.append(stacked_prediction)
    losses.append(loss_layer([targets[:, 0], mu, sigma, alpha]))
#     print("loss targets: ", targets[:, 0].type_spec.shape)
#     print("loss_layer: ", loss_layer([targets[:, 0], mu, sigma, alpha]).type_spec.shape)
    log_probabilities.append(gnll_metric(targets[:, 0], mu, sigma, alpha))
#     print("gnll_metric: ", gnll_metric(targets[:, 0], mu, sigma, alpha).type_spec.shape)
#     print("targets: ", targets.type_spec.shape)
    
    prediction = reshaper(stacked_prediction)
#     print("prediction: ", prediction.type_spec.shape)
#     print('=================================================================')
    
    for idx in range(1, config['N_FORWARD']):
        x = prediction
        x, hidden_state1, cell_state1 = lstm1(x, initial_state=[hidden_state1, cell_state1])
        x, hidden_state2, cell_state2 = lstm2(x, initial_state=[hidden_state2, cell_state2])
        x, hidden_state3, cell_state3 = lstm3(x, initial_state=[hidden_state3, cell_state3])
        x = K.concatenate([x, future_covs[:, idx, :]])
        x = dense1(x)
        x = dense2(x)
        
        # calculate the distribution parameters
        mu = mu_layer(x)
        sigma = sigma_layer(x)
        alpha = alpha_layer(x)
        
        stacked_prediction = tf.stack([mu, sigma, alpha], axis=1)
#         print("stacked_predictions: ", stacked_prediction.type_spec.shape)
        forecasts.append(stacked_prediction)
        losses.append(loss_layer([targets[:, idx], mu, sigma, alpha]))
#         print("loss_layer: ", loss_layer([targets[:, idx], mu, sigma, alpha]).type_spec.shape)
        log_probabilities.append(gnll_metric(targets[:, idx], mu, sigma, alpha))
#         print("gnll_metric: ", gnll_metric(targets[:, idx], mu, sigma, alpha).type_spec.shape)
        
        prediction = reshaper(stacked_prediction)
#         print("prediction: ", prediction.type_spec.shape)
#         print('=================================================================')
        
#     forecasts_output = tf.keras.layers.Lambda(lambda x: K.concatenate(x, axis=-1))(forecasts)
    forecasts_output = tf.stack(forecasts, axis=1)
    loss_output = tf.stack(losses, axis=1)
    log_probabilities_output = tf.stack(log_probabilities, axis=1)
    
    # reduce the values for easy reporting
    loss_output = tf.reduce_mean(loss_output, axis=1)
    log_probabilities_output = tf.reduce_mean(log_probabilities_output, axis=1)
    
#     print("forecasts_output: ", forecasts_output.type_spec.shape)
#     print("log probabilities: ", log_probabilities_output.type_spec.shape)
#     print("loss: ", loss_output.type_spec.shape)
    # consider multiplying by a decay rate function so that more recent forecasts are of higher importance
    
    model = tf.keras.models.Model(inputs=[inputs,future_covs,targets], outputs=loss_output)
    forecaster = tf.keras.models.Model(inputs=[inputs,future_covs], outputs=forecasts_output)
#     model.add_loss(identity)
    model.add_metric(log_probabilities_output, name="LogProbability")
    model.compile(loss=identity, optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"]))
    return model, forecaster

# def get_model():
#     model = MyModel()
#     model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(config["LR"]), metrics=['mae'])
#     model.predict(tf.ones(shape=(config['BATCH_SIZE'],config['N_PREV'],1)))
#     return model
    
test_model, _ = get_model()
# print(test_model.summary())
del test_model
gc.collect()

1392

## **Train Loop**

In [7]:
def train_model(model, train_ds, val_ds):
    reducer = tf.keras.callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.1, patience=2, mode='min', cooldown=1)
    stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)
    model.fit(train_ds,
              epochs=config["EPOCHS"], 
              callbacks=[reducer, stopper, WandbCallback()],
              validation_data=val_ds)
    return

def sample(mu, sigma, alpha, num_samples=200):
    indices = tf.random.categorical(
        logits=tf.log(alpha),
        num_samples=num_samples
    )
    sample_mu = tf.batch_gather(mu, indecies)
    sample_sigma = tf.batch_gather(sigma, indecies)
    normal_samples = K.random_normal(shape=K.shape(indecies))
    samples = sample_mu + sample_sigma * normal_samples
    print(tf.shape(samples))
    return samples

def sample_distribution(mu, sigma, alpha):
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Normal(
            loc=mu,       
            scale=sigma
        )
    )
    mean = gm.mean()
    std = gm.stddev()
    return mean.numpy(), std.numpy()

def test_model(forecaster, X_test, y_test, future_test, normalizer):
    predictions = forecaster.predict({'series':X_test, 'future_dates':future_test})
    print(predictions.shape)
    print(y_test.shape)
    
    mus = predictions[:, :, 0, :]
    sigmas = predictions[:, :, 1, :]
    alphas = predictions[:, :, 2, :]
    
    means, stds = sample_distribution(mus, sigmas, alphas)
    
    on_predictions = means
    over_predictions = means + stds
    under_predictions = means - stds
    
    inversed_predictions, inversed_over_sigma, inversed_under_sigma, inversed_actuals = [], [], [], []
    for time_idx in range(y_test.shape[1]):
        inversed_predictions.append(normalizer.inverse_transform(on_predictions[:,time_idx].reshape(-1,1)).reshape((-1)))
        inversed_under_sigma.append(normalizer.inverse_transform(under_predictions[:,time_idx].reshape(-1,1)).reshape((-1)))
        inversed_over_sigma.append(normalizer.inverse_transform(over_predictions[:,time_idx].reshape(-1,1)).reshape((-1)))
        inversed_actuals.append(normalizer.inverse_transform(y_test[:,time_idx,0].reshape(-1,1)).reshape((-1)))
    
    inversed_predictions = np.array(inversed_predictions).T
    inversed_under_sigma = np.array(inversed_under_sigma).T
    inversed_over_sigma = np.array(inversed_over_sigma).T
    inversed_actuals = np.array(inversed_actuals).T
    print("Inverses", inversed_predictions.shape, inversed_under_sigma.shape, inversed_over_sigma.shape, inversed_actuals.shape)
    
    forward_error = np.mean(np.abs(inversed_actuals - inversed_predictions), axis=-1)
    error = np.mean(forward_error)
    return error, forward_error, inversed_predictions, inversed_actuals, inversed_under_sigma, inversed_over_sigma

def record_forward_error(forward_errors, error, run):
    run.log({'test/error': error})
    run.log({'test/forward_error': wandb.plot.line_series(
        xs=[i+1 for i in range(forward_errors.shape[0])],
        ys=[forward_errors],
        keys=['Forward Horizon Errors'],
        title=f'Forward Horizon Errors (MAE: {error})',
    )})
        
def graph_predictions(predictions, actuals, dates, run, inversed_under_sigma, inversed_over_sigma):
    for look_ahead in [1, 12, 24, 36, 48]:
        if(look_ahead - 1 >= predictions.shape[1]):
            continue
        preds, true, time, under, over = predictions[:, look_ahead-1], actuals[:, look_ahead-1], dates[:, look_ahead-1], inversed_under_sigma[:, look_ahead-1], inversed_over_sigma[:, look_ahead-1]
        error = np.mean(np.abs(true - preds))
        run.log({f'test/forecast-{look_ahead}-hours-ahead': wandb.plot.line_series(
            xs=time,
            ys=[true, preds, under, over],
            keys=['Actuals', 'Predictions', 'Lower STD', 'Upper STD'],
            title=f'{look_ahead} Hour(s) Ahead Forecast (MAE: {error})',
            xname='week'
        )})

In [8]:
def train():
#     for fold in range(1, 4):
    for fold in [3,]:
        run = wandb.init(project="time-series-methods", entity="kmcguigan", group=f"{MODEL}-model", config=config, job_type="train")
        run.name = f'{MODEL}-fold-{fold}'
        (train_ds, val_ds, X_test, y_test, future_test, dates), normalizer = get_data(fold)
        model, forecaster = get_model()
        train_model(model, train_ds, val_ds)
        error, forward_errors, predictions, actuals, inversed_under_sigma, inversed_over_sigma = test_model(forecaster, X_test, y_test, future_test, normalizer)
        record_forward_error(forward_errors, error, run)
        graph_predictions(predictions, actuals, dates, run, inversed_under_sigma, inversed_over_sigma)
        run.finish()
        del model
        del train_ds
        del val_ds
        gc.collect()
    return

In [9]:
train()

Data Shape: (245876, 23), Reduced Shape (100000,)
Train shape: (84857, 120, 3, 1), (84857, 48, 6), (84857, 48, 1)
Val shape: (14975, 120, 3, 1), (14975, 48, 6), (14975, 48, 1)
Data Shape: (43390, 23), Reduced Shape (43390,)
Test shape: (43222, 120, 3, 1), (43222, 48, 6), (43222, 48, 1), (43222, 48)


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/50


2022-07-04 11:35:28.787194: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-04 11:35:29.091425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:48.703556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:48.920685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:49.156890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:49.421741: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:49.574371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113

2022-07-04 11:35:54.689233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:54.751648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:54.861962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:55.053849: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:55.110249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:55.213171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:35:55.403152: I tensorflow/core/grappler/optimizers/cust

...

2022-07-04 11:51:11.882939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:22.051864: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:22.653091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:23.100902: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:23.744919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:23.929607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:24.229386: I tensorflow/core/grappler/optimizers/cust

2022-07-04 11:51:42.566089: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:42.856009: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:43.539822: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:43.733982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:44.049907: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:44.668234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:51:44.926290: I tensorflow/core/grappler/optimizers/cust

2022-07-04 11:52:04.984693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:52:05.222310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:52:05.608653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:52:06.291618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:52:06.586997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:52:07.006995: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-04 11:52:07.686233: I tensorflow/core/grappler/optimizers/cust

166/166 [==============================] - 1294s 7s/step - loss: 1.4760 - LogProbability: 0.2750 - val_loss: 1.4047 - val_LogProbability: 0.2840 - lr: 0.0010 - _timestamp: 1656957324.0000 - _runtime: 1319.0000
Epoch 2/50
166/166 [==============================] - 2733s 17s/step - loss: 1.4037 - LogProbability: 0.2845 - val_loss: 1.4001 - val_LogProbability: 0.2884 - lr: 0.0010 - _timestamp: 1656960058.0000 - _runtime: 4053.0000
Epoch 3/50
166/166 [==============================] - 3612s 20s/step - loss: 1.3876 - LogProbability: 0.2897 - val_loss: 1.3581 - val_LogProbability: 0.2957 - lr: 0.0010 - _timestamp: 1656963670.0000 - _runtime: 7665.0000
Epoch 4/50
166/166 [==============================] - 1700s 10s/step - loss: 1.2953 - LogProbability: 0.3249 - val_loss: 1.1813 - val_LogProbability: 0.3706 - lr: 0.0010 - _timestamp: 1656965370.0000 - _runtime: 9365.0000
Epoch 5/50
 45/166 [=======>......................] - ETA: 28:24 - loss: 1.1744 - LogProbability: 0.3705

KeyboardInterrupt: 